In [ ]:
# 1. Read all GGPONC sentences
# 2. Run through compound splitter to produce Gertwol-like list
# 3. Create word frequencies from GGPONC sentences
# 4. Apply spaCy POS Tagger to all raw sentences and write to XML
# 5. Run tool: https://www.cl.uzh.ch/en/texttechnologies/publications/downloads.html